In [1]:
# System Libraries
from tensorflow import keras
import tensorflow as tf
import numpy as np
import importlib
import sys

sys.path.append("../")

# User Libraries
from modules import Chapter_01
from modules import Common

# Reload Libraries
importlib.reload(Chapter_01)
importlib.reload(Common)

<module 'modules.Common' from '/Users/siddiqui/Projects/concepts/concepts-ai/practices/../modules/Common.py'>

### **Single IO — Sequential**

In [2]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(3,)))
model.add(keras.layers.Dense(units=64, activation=tf.nn.relu))
model.add(keras.layers.Dense(units=10, activation=tf.nn.softmax))

### **Single IO — Functional**

In [3]:
inputs = keras.layers.Input(shape=(3,))
features = keras.layers.Dense(units=64, activation=tf.nn.relu)(inputs)
outputs = keras.layers.Dense(units=10, activation=tf.nn.softmax)(features)
model = keras.Model(inputs=inputs, outputs=outputs)

### **Multi IO — Functional**

In [4]:
num_samples = 100
num_words = 10
num_tags = 5
num_priorities = 1
num_departments = 4
num_difficulties = 3

In [5]:
title = keras.layers.Input(shape=(num_words,))
text = keras.layers.Input(shape=(num_words,))
tag = keras.layers.Input(shape=(num_tags,))
features = keras.layers.Concatenate()([title, text, tag])
features = keras.layers.Dense(units=64, activation=tf.nn.relu)(features)
features = keras.layers.Dropout(rate=0.5)(features)
priority = keras.layers.Dense(units=num_priorities, activation=tf.nn.sigmoid, name="O_priority")(features)
department = keras.layers.Dense(units=num_departments, activation=tf.nn.softmax, name="O_department")(features)
model = keras.Model(inputs=[title, text, tag], outputs=[priority, department])

In [6]:
features = model.layers[5].output
difficulty = keras.layers.Dense(units=num_difficulties, activation=tf.nn.softmax, name="O_difficulty")(features)
model = keras.Model(inputs=[title, text, tag], outputs=[priority, department, difficulty])

In [7]:
I_title = np.random.randint(0, num_words, size=(num_samples, num_words))
I_text = np.random.randint(0, num_words, size=(num_samples, num_words))
I_tag = np.random.randint(0, num_tags, size=(num_samples, num_tags))
O_priority = np.random.random(size=(num_samples, num_priorities))
O_department = np.random.randint(0, num_departments, size=(num_samples, num_departments))
O_difficulty = np.random.randint(0, num_difficulties, size=(num_samples, num_difficulties))

In [8]:
model.compile(
    loss={
        "O_priority": keras.losses.MeanSquaredError(),
        "O_department": keras.losses.CategoricalCrossentropy(),
        "O_difficulty": keras.losses.CategoricalCrossentropy(),
    },
    metrics={
        "O_priority": keras.metrics.MeanAbsoluteError(),
        "O_department": keras.metrics.CategoricalAccuracy(),
        "O_difficulty": keras.metrics.CategoricalAccuracy(),
    },
    optimizer=keras.optimizers.legacy.RMSprop(),
)

In [9]:
history = model.fit(
    x=[I_title, I_text, I_tag],
    y=[O_priority, O_department, O_difficulty],
    verbose=False,
    epochs=2,
    validation_split=0.3,
)
model.evaluate(
    x=[I_title, I_text, I_tag],
    y=[O_priority, O_department, O_difficulty],
    verbose=False,
)
predictions = model.predict(x=[I_title, I_text, I_tag], verbose=False)

### **Multi IO — Subclass**

In [10]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.concat_layer = keras.layers.Concatenate()
        self.mixing_layer = keras.layers.Dense(units=64, activation=tf.nn.relu)
        self.priority_scorer = keras.layers.Dense(units=num_priorities, activation=tf.nn.sigmoid, name="O_priority")
        self.department_classifier = keras.layers.Dense(
            units=num_departments, activation=tf.nn.softmax, name="O_department"
        )

    def call(self, inputs):
        title, text, tag = inputs
        features = self.concat_layer([title, text, tag])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

In [11]:
model = CustomModel()
model.compile(
    loss=[keras.losses.MeanSquaredError(), keras.losses.CategoricalCrossentropy()],
    metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.CategoricalAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)

In [12]:
history = model.fit(
    x=[I_title, I_text, I_tag],
    y=[O_priority, O_department],
    verbose=False,
    epochs=2,
    validation_split=0.3,
)
model.evaluate(
    x=[I_title, I_text, I_tag],
    y=[O_priority, O_department],
    verbose=False,
)
predictions = model.predict(x=[I_title, I_text, I_tag], verbose=False)

### **Mnist — Functional**

In [13]:
(x_train, y_train), (x_test, y_test) = Chapter_01.prepare()
inputs = keras.layers.Input(shape=(x_train.shape[1]))
features = keras.layers.Dense(units=256, activation=tf.nn.relu)(inputs)
features = keras.layers.Dropout(rate=0.5)(features)
outputs = keras.layers.Dense(units=10, activation=tf.nn.softmax)(features)
model = keras.Model(inputs=inputs, outputs=outputs)

In [14]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)

In [15]:
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=5,
    batch_size=5000,
    verbose=False,
)
evaluation = model.evaluate(x=x_test, y=y_test, verbose=False)
y_pred = model.predict(x=x_test, verbose=False)

### **Custom Metrics**

In [16]:
class RootMeanSquaredError(keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.num_samples = self.add_weight(name="num_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        num_samples = tf.shape(y_pred)[0]
        self.mse_sum.assign_add(mse)
        self.num_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.num_samples, np.float32))

    def reset_state(self):
        self.mse_sum.assign(0.0)
        self.num_samples.assign(0)

In [17]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(), RootMeanSquaredError()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)

In [18]:
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=5,
    batch_size=5000,
    verbose=False,
)
evaluation = model.evaluate(x=x_test, y=y_test, verbose=False)
y_pred = model.predict(x=x_test, verbose=False)
print(history.history.keys())

dict_keys(['loss', 'sparse_categorical_accuracy', 'rmse', 'val_loss', 'val_sparse_categorical_accuracy', 'val_rmse'])


### **Builtin Callbacks**

In [19]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="model.keras",
        monitor="val_sparse_categorical_accuracy",
        save_best_only=True,
    ),
]

In [20]:
model = Chapter_01.get_function_model()
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=5000,
    callbacks=[callbacks],
    verbose=False,
)

### **Custom Callbacks**

In [29]:
class CustomCallback(keras.callbacks.Callback):
    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass


callbacks = CustomCallback()

In [30]:
model = Chapter_01.get_function_model()
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=2,
    batch_size=52000,
    callbacks=[callbacks],
    verbose=False,
)

### **Save and Load Model**

In [23]:
model.save("model.keras")
model = keras.models.load_model("model.keras")

### **TensorBoard**

In [24]:
model = Chapter_01.get_function_model()
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=10,
    batch_size=12000,
    callbacks=[keras.callbacks.TensorBoard(log_dir="")],
    verbose=False,
)

### **Low Level Metrics**

In [25]:
metric = keras.metrics.SparseCategoricalAccuracy()
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
targets = [0, 1, 2]
targets_onehot = tf.one_hot(targets, depth=tf.shape(predictions)[1])
metric.update_state(targets, predictions)
metric.result()
metric.reset_state()

In [26]:
metric = keras.metrics.Mean()
values = [1, 3, 9]
metric.update_state(values)
metric.result()
metric.reset_state()